In [116]:
import pandas as pd
import numpy as np
import os

import regex as re
import altair as alt
import matplotlib.pyplot as plt
import squarify

from datetime import datetime
from datetime import timedelta

In [ ]:
"This file was used to create the improved dataset used throughout the report. We improved on the prior version \
by adding a Province column and a 'Total employees' column, which allowed us to create visualizations without continuously \
re-querying for this information"

In [117]:
#os.chdir('w2020-data599-capstone-projects-statistics-canada-covid-19')
#os.chdir('../../..')
os.getcwd()

'C:\\Users\\ebaxt\\Documents\\Masters\\capstone\\w2020-data599-capstone-projects-statistics-canada-covid-19'

In [118]:
cews = pd.read_csv("CEWS_with_subset_labels.csv")
# First column is just the index again
cews = cews.drop("Unnamed: 0", axis = 1)
# Datetime
cews.Period = pd.to_datetime(cews.Period, infer_datetime_format=True)

empByCSD = pd.read_csv("more_data/empByCSD.csv")

empByCMA = pd.read_csv("more_data/empByCMA.csv")

In [119]:
# Create Province Column in cews

cews["ProvinceCode"] = cews["RegionCode"].str.extract(r'(^\d{2})')

# Get province names

temp2 = cews.query("GeoAggregation == 'Province' and IndustryAggregation == 'All industries'")
temp2 = temp2.filter(["RegionCode", "Region"])
temp2 = temp2.drop_duplicates()
cews = cews.merge(temp2, how = "left", left_on = "ProvinceCode", right_on = "RegionCode")

cews = cews.rename(columns={"Region_y": "Province", "Region_x": "Region", "RegionCode_x": "RegionCode"})
cews = cews.drop("RegionCode_y", axis = 1)


In [120]:
# Employee Counts by Census subdivision

# FIlter for all sex
empByCSD = empByCSD[empByCSD["DIM: Sex (3)"] == "Total - Sex"]
# Filter by job type
empByCSD = empByCSD[empByCSD["Member ID: Occupation - National Occupational Classification (NOC) 2016 (11)"] == 1]
empByCSD = empByCSD.rename(columns={"DIM: Industry - North American Industry Classification System (NAICS) 2012 (21)": "Industry", 
                                    "Dim: Place of work status (5): Member ID: [1]: Total - Place of work status (Note: 3)": "TotalEmployees",
                                   "GEO_CODE (POR)": "RegionCode"})
empByCSD = empByCSD.filter(["Industry", "RegionCode", "TotalEmployees"])

In [121]:
# Code comes before the first space
def extractNIAC(industry):
    return industry[0: industry.find(' ')]

empByCSD["IndustryCode"] = empByCSD["Industry"].apply(extractNIAC)
empByCSD = empByCSD.drop("Industry", axis = 1)

In [122]:
# Change region code 1 to TOTAL
# Change industrycode Total to TOTAL
empByCSD["IndustryCode"] = empByCSD["IndustryCode"].replace("Total","TOTAL")
empByCSD["RegionCode"] = empByCSD["RegionCode"].astype(str)
empByCSD["RegionCode"] = empByCSD["RegionCode"].replace("1", "TOTAL")

In [123]:
# Employee counts by Census agglomeration DB

empByCMA = empByCMA[empByCMA["DIM: Sex (3)"] == "Total - Sex"]
empByCMA = empByCMA[empByCMA["Member ID: Occupation - National Occupational Classification (NOC) 2016 (11)"] == 1]
empByCMA = empByCMA.rename(columns={"DIM: Industry - North American Industry Classification System (NAICS) 2012 (21)": "Industry", 
                                    "Dim: Place of work status (5): Member ID: [1]: Total - Place of work status (Note: 3)": "TotalEmployees",
                                   "ALT_GEO_CODE": "RegionCode"})

empByCMA = empByCMA.filter(["Industry", "RegionCode", "TotalEmployees"])

empByCMA["IndustryCode"] = empByCMA["Industry"].apply(extractNIAC)
empByCMA = empByCMA.drop("Industry", axis = 1)

# Change region code 1 to TOTAL
# Change industrycode Total to TOTAL
empByCMA["IndustryCode"] = empByCMA["IndustryCode"].replace("Total","TOTAL")
empByCMA["RegionCode"] = empByCMA["RegionCode"].astype(str)
empByCMA["RegionCode"] = empByCMA["RegionCode"].replace("1", "TOTAL")

In [124]:
# These two datasets both include countrywide and provincial data, which we drop in empByCMA
empByCMA = empByCMA[empByCMA["RegionCode"].str.contains("^\d{5}$")]
emp = empByCSD.append(empByCMA)

In [125]:
emp.query("RegionCode == '24462'").sort_values("IndustryCode")

,RegionCode,TotalEmployees,IndustryCode
33958,24462,9145,11
33959,24462,2610,21
33960,24462,14325,22
33961,24462,102040,23
33962,24462,203200,31-33
33963,24462,94630,41
33964,24462,245285,44-45
33965,24462,101675,48-49
33966,24462,69895,51
33967,24462,99645,52


In [126]:
cews = cews.merge(emp, how = 'left', on = ["RegionCode", "IndustryCode"])

In [127]:
# Test to make sure this seems right
cews.query("IndustryAggregation == 'All industries' and GeoAggregation == 'Province'").sample(20)

,Period,RegionCode,Region,GeographicClassification,CensusLevel,IndustryCode,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,GeoAggregation,IndustryAggregation,ProvinceCode,Province,TotalEmployees
70292,2020-03-15,61,Northwest Territories,Not applicable,Not applicable,TOTAL,All Industries,330.0,1.320100e+07,4992.0,15.0,Province,All industries,61,Northwest Territories,21415.0
6736,2020-03-15,13,New Brunswick,Not applicable,Not applicable,TOTAL,All Industries,5865.0,1.050720e+08,52785.0,540.0,Province,All industries,13,New Brunswick,339050.0
437670,2020-08-02,62,Nunavut,Not applicable,Not applicable,TOTAL,All Industries,110.0,3.958000e+06,2475.0,0.0,Province,All industries,62,Nunavut,12820.0
225432,2020-06-07,10,Newfoundland and Labrador,Not applicable,Not applicable,TOTAL,All Industries,3770.0,9.497700e+07,40189.0,470.0,Province,All industries,10,Newfoundland and Labrador,216705.0
296085,2020-06-07,61,Northwest Territories,Not applicable,Not applicable,TOTAL,All Industries,290.0,1.162400e+07,4157.0,25.0,Province,All industries,61,Northwest Territories,21415.0
223973,2020-05-10,60,Yukon,Not applicable,Not applicable,TOTAL,All Industries,400.0,7.737000e+06,3398.0,55.0,Province,All industries,60,Yukon,19785.0
502262,2020-08-30,60,Yukon,Not applicable,Not applicable,TOTAL,All Industries,290.0,3.967000e+06,3317.0,20.0,Province,All industries,60,Yukon,19785.0
182266,2020-05-10,35,Ontario,Not applicable,Not applicable,TOTAL,All Industries,121905.0,3.771701e+09,1546311.0,16115.0,Province,All industries,35,Ontario,6612150.0
224787,2020-05-10,62,Nunavut,Not applicable,Not applicable,TOTAL,All Industries,110.0,8.045000e+06,2812.0,10.0,Province,All industries,62,Nunavut,12820.0
295676,2020-06-07,60,Yukon,Not applicable,Not applicable,TOTAL,All Industries,345.0,7.162000e+06,3383.0,35.0,Province,All industries,60,Yukon,19785.0


In [128]:
cews.to_csv("cews_updated.csv", index = False)